<a href="https://colab.research.google.com/github/robert-shepherd/fpl/blob/main/Project_1_creating_model_history_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating model history file


---

The purpose of this script is to combine data from previous FPL seasons (18/19 and 19/20) with the latest data available for this season (GW11 for 20/21).

Data sources:
* Latest season: FPL API: https://fantasy.premierleague.com/api
* Historic seasons: User Github: https://github.com/vaastav/Fantasy-Premier-League

Output:
* Data is downloaded and subsequently saved to: https://raw.githubusercontent.com/robert-shepherd/fpl/main/fpl_combined_data.csv


In [ ]:
# Loading libraries
import pandas as pd
import requests
from google.colab import files

## Reading in data from the API for the current season


### Create look up files from static API

In [ ]:
# Import data from API
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [ ]:
# Create a dataframe of live elements
elements = pd.DataFrame(json['elements'])
elements.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,0.0,0,Mesut,0.0,1,False,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,37605.jpg,0.0,Özil,0.6,False,None,n,1,3,0,3405,5,52944,97,0.0,0.0,Özil,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,616,247,616,247,616,247,616,247,NaN,,NaN,,NaN,
1,0.0,0.0,39476,0,0,-2,2,0,2,0.0,0.0,0,Sokratis,0.0,2,False,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,48,39476.jpg,0.0,Papastathopoulos,0.1,False,None,n,1,3,0,10255,5,18840,20,0.0,0.0,Sokratis,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,574,212,565,212,553,208,577,212,NaN,,NaN,,NaN,
2,75.0,75.0,41270,0,0,-1,1,0,2,1.0,0.6,0,David,0.3,3,False,Illness - 75% chance of playing,2020-12-26T18:00:15.638627Z,54,41270.jpg,1.1,Luiz Moreira Marinho,0.7,False,None,d,1,3,9,39083,42,89267,883,0.1,1.7,David Luiz,479,0,0,0,9,0,0,0,0,0,0,0,70,99.4,25.8,40.0,16.5,260,100,296,93,259,79,295,97,NaN,,4.0,,NaN,
3,100.0,100.0,54694,0,0,-6,6,1,3,3.5,3.0,3,Pierre-Emerick,2.5,4,False,,2020-12-19T20:30:20.434896Z,114,54694.jpg,3.6,Aubameyang,7.3,False,None,a,1,3,50,499424,5586,3143022,6225,0.2,4.4,Aubameyang,1256,3,1,4,16,1,0,0,2,0,0,4,146,191.2,208.8,401.0,80.1,147,54,61,41,27,14,38,24,NaN,,NaN,,1.0,
4,100.0,100.0,58822,0,0,-4,4,0,2,1.2,0.7,0,Cédric,0.2,5,False,,2020-09-23T09:00:14.881983Z,46,58822.jpg,1.0,Soares,0.2,False,None,a,1,3,1,3887,39,34544,129,0.0,0.2,Cédric,5,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,561,208,549,208,532,202,564,208,NaN,,NaN,,NaN,


In [ ]:
# Create a lookup from element to element_type
element_type_ref = elements[['id','element_type']]
element_type_ref.columns = ['element','element_type']
element_type_ref.head()

,element,element_type
0,1,3
1,2,2
2,3,2
3,4,3
4,5,2


In [ ]:
# Create a dataframe of element types to get position
element_types = pd.DataFrame(json['element_types'])
element_types.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],71
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],221
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],247
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],77


In [ ]:
# Extracting position from the element_types data
position = element_types[['id','singular_name_short']]
position.columns = ['element_type','position']
position.head()

,element_type,position
0,1,GKP
1,2,DEF
2,3,MID
3,4,FWD


In [ ]:
# Create a dataframe of teams to get opponent strength
teams = pd.DataFrame(json['teams'])
teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1190,1210,1170,1210,1190,1200,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1150,1160,1150,1150,1180,1210,2
2,36,0,None,3,0,Brighton,0,0,0,BHA,3,None,False,0,1080,1100,1150,1180,1090,1100,131
3,90,0,None,4,0,Burnley,0,0,0,BUR,2,None,False,0,1050,1080,1120,1190,1010,1030,43
4,8,0,None,5,0,Chelsea,0,0,0,CHE,4,None,False,0,1260,1280,1240,1280,1270,1310,4


In [ ]:
# Extracting opponent_strength from the teams data
opponents = teams[['id','strength']]
opponents.columns = ['opponent_team','opponent_strength']
opponents.head()

,opponent_team,opponent_strength
0,1,4
1,2,3
2,3,3
3,4,2
4,5,4


### Looping over the elements API to create a history for current season

In [ ]:
# Loop through history and create a file
for x in elements.index :
    element_id = elements.id[x]
    url = f'https://fantasy.premierleague.com/api/element-summary/{element_id}/'
    r = requests.get(url)
    json = r.json()
    json_history = pd.DataFrame(json['history'])
    
    if x == 0 :
        history = json_history
    else : 
        history = history.append(json_history)


In [ ]:
# Checking records
history.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,1,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126
3,1,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971
4,1,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840


In [ ]:
# Checking shape
history.shape

(9125, 31)

In [ ]:
# Checking columns and column types
history.dtypes

element               int64
fixture               int64
opponent_team         int64
total_points          int64
was_home               bool
kickoff_time         object
team_h_score          int64
team_a_score          int64
round                 int64
minutes               int64
goals_scored          int64
assists               int64
clean_sheets          int64
goals_conceded        int64
own_goals             int64
penalties_saved       int64
penalties_missed      int64
yellow_cards          int64
red_cards             int64
saves                 int64
bonus                 int64
bps                   int64
influence            object
creativity           object
threat               object
ict_index            object
value                 int64
transfers_balance     int64
selected              int64
transfers_in          int64
transfers_out         int64
dtype: object

In [ ]:
# Merging element type reference
history_type = pd.merge(history,element_type_ref,on='element')
history_type.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,element_type
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0,3
1,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823,3
2,1,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126,3
3,1,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971,3
4,1,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840,3


In [ ]:
# Merging position type reference
history_pos = pd.merge(history_type,position,on='element_type')
history_pos.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,element_type,position
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0,3,MID
1,1,9,19,0,True,2020-09-19T19:00:00Z,2,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823,3,MID
2,1,23,11,0,False,2020-09-28T19:00:00Z,3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126,3,MID
3,1,29,15,0,True,2020-10-04T13:00:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971,3,MID
4,1,44,12,0,False,2020-10-17T16:30:00Z,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840,3,MID


In [ ]:
# Merging opposition difficulty
df_2021 = pd.merge(history_pos,opponents,on='opponent_team')

In [ ]:
# Checking 2021 data
df_2021.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,element_type,position,opponent_strength
0,1,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,70,0,76656,0,0,3,MID,2
1,4,2,8,7,False,2020-09-12T11:30:00Z,0,3,1,90,1,0,1,0,0,0,0,1,0,0,0,19,36.6,15.3,54.0,10.6,120,0,2823465,0,0,3,MID,2
2,9,2,8,3,False,2020-09-12T11:30:00Z,0,3,1,77,0,0,1,0,0,0,0,0,0,0,0,11,6.8,17.2,21.0,4.5,55,0,42909,0,0,3,MID,2
3,18,2,8,1,False,2020-09-12T11:30:00Z,0,3,1,15,0,0,0,0,0,0,0,0,0,0,0,3,2.0,15.3,6.0,2.3,80,0,132947,0,0,3,MID,2
4,19,2,8,0,False,2020-09-12T11:30:00Z,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,50,0,15039,0,0,3,MID,2


In [ ]:
# Checking 2021 shape
df_2021.shape

(9125, 34)

In [ ]:
# Checking 2021 data types
df_2021.dtypes
# Note: some numeric fields set as objects, corrected below

element               int64
fixture               int64
opponent_team         int64
total_points          int64
was_home               bool
kickoff_time         object
team_h_score          int64
team_a_score          int64
round                 int64
minutes               int64
goals_scored          int64
assists               int64
clean_sheets          int64
goals_conceded        int64
own_goals             int64
penalties_saved       int64
penalties_missed      int64
yellow_cards          int64
red_cards             int64
saves                 int64
bonus                 int64
bps                   int64
influence            object
creativity           object
threat               object
ict_index            object
value                 int64
transfers_balance     int64
selected              int64
transfers_in          int64
transfers_out         int64
element_type          int64
position             object
opponent_strength     int64
dtype: object

In [ ]:
#Casting latest season influence, creativity, threat and ict_index to float
df_2021['influence'] = df_2021['influence'].astype(float)
df_2021['creativity'] = df_2021['creativity'].astype(float)
df_2021['threat'] = df_2021['threat'].astype(float)
df_2021['ict_index'] = df_2021['ict_index'].astype(float)
df_2021.dtypes

element                int64
fixture                int64
opponent_team          int64
total_points           int64
was_home                bool
kickoff_time          object
team_h_score           int64
team_a_score           int64
round                  int64
minutes                int64
goals_scored           int64
assists                int64
clean_sheets           int64
goals_conceded         int64
own_goals              int64
penalties_saved        int64
penalties_missed       int64
yellow_cards           int64
red_cards              int64
saves                  int64
bonus                  int64
bps                    int64
influence            float64
creativity           float64
threat               float64
ict_index            float64
value                  int64
transfers_balance      int64
selected               int64
transfers_in           int64
transfers_out          int64
element_type           int64
position              object
opponent_strength      int64
dtype: object

## Reading in data for previous seasons

### Preparing gameweek data

In [ ]:
# Reading in gameweek data from previous seasons
url_1819 = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/merged_gw.csv'
url_1920 = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/merged_gw.csv'

gw_1819 = pd.read_csv(url_1819, encoding='latin-1')
gw_1920 = pd.read_csv(url_1920, encoding='latin-1')

In [ ]:
# Checking 1819 data
gw_1819.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell_402,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0,0,0,55,False,0,0,1
1,Aaron_Lennon_83,0,22,0,1,0,6,1,1,17,12.3,0,0,83,0,0,8,1,0,0,3.9,83,10.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,90,0,1,16,0,0,0,0,2,0,1,0,15138,1,2,0,0,0,17.0,3,0,0,0,50,False,0,0,1
2,Aaron_Mooy_199,0,51,0,0,0,24,0,2,40,18.2,1,0,199,0,0,4,1,3,0,3.8,199,20.2,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,6,0,0,0,0,11,0,1,0,192110,1,6,0,3,0,0.0,2,0,0,0,55,True,0,0,1
3,Aaron_Ramsey_14,0,11,0,0,0,7,0,0,7,10.8,1,0,14,0,0,1,0,1,0,2.9,14,9.4,1,2018-08-12T15:00:00Z,12 Aug 16:00,0,0,53,2,0,13,0,0,0,0,1,0,1,0,60423,0,2,0,2,0,9.0,1,0,0,0,75,True,0,0,1
4,Aaron_Wan-Bissaka_145,1,29,1,0,3,38,1,11,19,14.0,2,0,145,0,0,3,0,0,0,6.0,145,46.0,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,9,0,0,0,0,7,0,1,0,652304,2,0,0,2,0,0.0,12,0,0,0,40,False,0,0,1


In [ ]:
# Checking 1819 shape
gw_1819.shape

(21790, 56)

In [ ]:
# Checking 1920 data
gw_1920.head()

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,1.1,9.0,2019-08-10T11:30:00Z,90,11,0,0,0,0,1,0,23399,5.0,0.0,0.0,0,0,0,0,50,True,0,1
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,0.2,2.0,2019-08-10T14:00:00Z,6,16,0,0,0,0,1,0,8105,0.0,3.0,0.0,1,0,0,0,50,True,0,1
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,0.0,0.0,2019-08-10T14:00:00Z,0,18,0,0,0,0,1,0,16261,3.0,0.0,0.0,0,0,0,0,50,False,0,1
3,Aaron_Ramsdale_494,0,0,11,0,0.0,494,2,1,0,1.0,9.8,2019-08-10T14:00:00Z,90,15,0,0,0,0,1,2,3091,1.0,1.0,0.0,2,0,0,0,45,True,0,1
4,Aaron_Wan-Bissaka_122,0,2,34,1,16.1,122,9,0,0,4.9,30.4,2019-08-11T15:30:00Z,90,6,0,0,0,0,1,0,1879259,0.0,4.0,2.0,8,0,0,0,55,True,0,1


In [ ]:
# Checking 1920 shape
gw_1920.shape

(22560, 33)

### Preparing player data

In [ ]:
# Reading in player data from previous seasons
player_url_1819 = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/players_raw.csv'
player_url_1920 = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv'

player_1819 = pd.read_csv(player_url_1819, encoding='latin-1')
player_1920 = pd.read_csv(player_url_1920, encoding='latin-1')

In [ ]:
# Create a lookup from element to element_type
player_1819_ref = player_1819[['id','element_type']]
player_1819_ref.columns = ['element','element_type']
player_1819_ref.head()

,element,element_type
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [ ]:
# Create a lookup from element to element_type
player_1920_ref = player_1920[['id','element_type']]
player_1920_ref.columns = ['element','element_type']
player_1920_ref.head()

,element,element_type
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2


### Preparing opponent data

In [ ]:
# Reading in teams data from 1819 season
teams_url_1920 = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/teams.csv'
teams_1920 = pd.read_csv(teams_url_1920, encoding='latin-1')
teams_1920.head()

# Note: teams file for 1819 unavailable so needs to be read from historic JSON file

,code,draw,form,id,loss,name,played,points,position,pulse_id,short_name,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,NaN,1,0,Arsenal,0,0,0,1,ARS,4,1170,1170,1200,1150,1240,1180,NaN,False,0
1,7,0,NaN,2,0,Aston Villa,0,0,0,2,AVL,2,980,970,1040,1000,1050,1020,NaN,False,0
2,91,0,NaN,3,0,Bournemouth,0,0,0,127,BOU,2,1030,990,1050,1000,1020,1020,NaN,False,0
3,36,0,NaN,4,0,Brighton,0,0,0,131,BHA,2,1100,1100,1030,1040,1010,1050,NaN,False,0
4,90,0,NaN,5,0,Burnley,0,0,0,43,BUR,3,1070,1130,1110,970,1180,1110,NaN,False,0


In [ ]:
# Extracting opponent_strength from the teams data
opponents_1920 = teams_1920[['id','strength']]
opponents_1920.columns = ['opponent_team','opponent_strength']
opponents_1920.head()

,opponent_team,opponent_strength
0,1,4
1,2,2
2,3,2
3,4,2
4,5,3


In [ ]:
# Import data from raw JSON file
url = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/raw.json'
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['stats', 'next-event', 'teams', 'element_types', 'elements', 'last-entry-event', 'stats_options', 'game-settings', 'current-event', 'next_event_fixtures', 'phases', 'events', 'total-players'])

In [ ]:
# Create a dataframe of teams to get opponent strength
teams_1819 = pd.DataFrame(json['teams'])
teams_1819.head()

,strength_defence_home,team_division,position,strength,strength_overall_home,strength_attack_home,current_event_fixture,played,strength_attack_away,unavailable,win,next_event_fixture,draw,id,strength_overall_away,strength_defence_away,code,name,loss,form,short_name,points,link_url
0,1310,1,0,4,1260,1240,"[{'opponent': 4, 'event_day': 1, 'month': 5, '...",0,1270,False,0,[],0,1,1320,1340,3,Arsenal,0,None,ARS,0,
1,1120,1,0,3,1030,1040,"[{'opponent': 7, 'event_day': 1, 'month': 5, '...",0,1100,False,0,[],0,2,1130,1130,91,Bournemouth,0,None,BOU,0,
2,1010,1,0,2,1030,1040,"[{'opponent': 13, 'event_day': 1, 'month': 5, ...",0,1140,False,0,[],0,3,1050,1070,36,Brighton,0,None,BHA,0,
3,1000,1,0,3,1070,990,"[{'opponent': 1, 'event_day': 1, 'month': 5, '...",0,1030,False,0,[],0,4,1100,1040,90,Burnley,0,None,BUR,0,
4,1020,1,0,2,1030,1030,"[{'opponent': 14, 'event_day': 1, 'month': 5, ...",0,1060,False,0,[],0,5,1080,1090,97,Cardiff,0,None,CAR,0,


In [ ]:
# Extracting opponent_strength from the teams data
opponents_1819 = teams_1819[['id','strength']]
opponents_1819.columns = ['opponent_team','opponent_strength']
opponents_1819.head()

,opponent_team,opponent_strength
0,1,4
1,2,3
2,3,2
3,4,3
4,5,2


### Merging previous season data

#### 1819

In [ ]:
# Merging element type reference
history_1819_type = pd.merge(gw_1819,player_1819_ref,on='element')

# Merging position type reference
history_1819_pos = pd.merge(history_1819_type,position,on='element_type')
history_1819_pos.head()
# Note: position references are consistent across seasons

# Merging opposition difficulty
df_1819 = pd.merge(history_1819_pos,opponents_1819,on='opponent_team')

In [ ]:
# Checking 1819 data
df_1819.head()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,element_type,position,opponent_strength
0,Aaron_Cresswell_402,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0,0,0,55,False,0,0,1,2,DEF,5
1,Aaron_Cresswell_402,0,19,0,0,0,7,0,4,10,0.6,0,0,402,0,0,250,0,1,0,1.8,13716,7.6,0,2019-02-04T20:00:00Z,04 Feb 20:00,0,0,90,0,0,12,0,0,0,0,7,0,25,0,32970,1,0,2,1,1,10.0,2,-787,27,814,50,True,0,0,25,2,DEF,5
2,Aaron_Wan-Bissaka_145,0,21,0,0,0,-1,0,3,15,12.3,4,0,145,0,0,15,1,2,0,2.9,670,12.4,1,2018-08-20T19:00:00Z,20 Aug 20:00,0,0,74,0,0,12,0,0,0,0,2,1,2,0,1110097,3,0,0,2,0,4.0,-2,282492,296156,13664,41,True,0,0,2,2,DEF,5
3,Aaron_Wan-Bissaka_145,0,27,0,0,0,15,0,8,17,0.8,0,0,145,0,0,225,0,4,0,1.9,12250,17.8,0,2019-01-19T15:00:00Z,19 Jan 15:00,0,0,90,0,0,12,0,0,0,0,7,0,23,0,2025265,3,3,0,3,4,0.0,0,21257,76053,54796,45,False,0,0,23,2,DEF,5
4,Adam_Masina_386,0,41,0,0,0,12,0,3,28,19.9,1,0,386,0,0,128,1,3,0,3.2,6858,11.8,1,2018-11-24T15:00:00Z,24 Nov 15:00,0,0,90,0,1,12,0,0,0,0,4,0,13,0,6017,0,1,0,3,0,0.0,1,-768,166,934,43,True,0,0,13,2,DEF,5


In [ ]:
# Checking 1819 shape
df_1819.shape

(21790, 59)

In [ ]:
# Checking 1819 data types
df_1819.dtypes

name                                object
assists                              int64
attempted_passes                     int64
big_chances_created                  int64
big_chances_missed                   int64
bonus                                int64
bps                                  int64
clean_sheets                         int64
clearances_blocks_interceptions      int64
completed_passes                     int64
creativity                         float64
dribbles                             int64
ea_index                             int64
element                              int64
errors_leading_to_goal               int64
errors_leading_to_goal_attempt       int64
fixture                              int64
fouls                                int64
goals_conceded                       int64
goals_scored                         int64
ict_index                          float64
id                                   int64
influence                          float64
key_passes 

#### 1920

In [ ]:
# Merging element type reference
history_1920_type = pd.merge(gw_1920,player_1920_ref,on='element')

# Merging position type reference
history_1920_pos = pd.merge(history_1920_type,position,on='element_type')
history_1920_pos.head()
# Note: position references are consistent across seasons

# Merging opposition difficulty
df_1920 = pd.merge(history_1920_pos,opponents_1920,on='opponent_team')

In [ ]:
# Checking 1920 data
df_1920.head()

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,element_type,position,opponent_strength
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,1.1,9.0,2019-08-10T11:30:00Z,90,11,0,0,0,0,1,0,23399,5.0,0.0,0.0,0,0,0,0,50,True,0,1,2,DEF,5
1,Aaron_Cresswell_376,0,0,13,0,11.4,376,256,2,0,3.2,20.4,2020-02-19T19:30:00Z,90,11,0,0,0,0,26,0,98411,0.0,2.0,0.0,1,-10379,1090,11469,47,False,0,26,2,DEF,5
2,Aaron_Wan-Bissaka_122,0,1,24,0,10.9,122,155,1,0,5.2,41.4,2019-12-07T17:30:00Z,90,11,0,0,0,0,16,0,1032199,2.0,1.0,0.0,3,-37726,4386,42112,53,False,0,16,2,DEF,5
3,Aaron_Wan-Bissaka_122,0,1,25,1,10.9,122,287,0,0,4.6,33.4,2020-03-08T16:30:00Z,90,11,0,0,0,0,29,0,1039230,0.0,2.0,2.0,7,-34971,9620,44591,53,True,0,29,2,DEF,5
4,Adam_Masina_355,0,0,0,0,0.0,355,57,0,0,0.0,0.0,2019-09-21T14:00:00Z,0,11,0,0,0,0,6,0,4425,0.0,8.0,0.0,0,-158,26,184,44,False,0,6,2,DEF,5


In [ ]:
# Checking 1920 shape
df_1920.shape

(22560, 36)

In [ ]:
# Checking 1920 data types
df_1920.dtypes

name                  object
assists                int64
bonus                  int64
bps                    int64
clean_sheets           int64
creativity           float64
element                int64
fixture                int64
goals_conceded         int64
goals_scored           int64
ict_index            float64
influence            float64
kickoff_time          object
minutes                int64
opponent_team          int64
own_goals              int64
penalties_missed       int64
penalties_saved        int64
red_cards              int64
round                  int64
saves                  int64
selected               int64
team_a_score         float64
team_h_score         float64
threat               float64
total_points           int64
transfers_balance      int64
transfers_in           int64
transfers_out          int64
value                  int64
was_home                bool
yellow_cards           int64
GW                     int64
element_type           int64
position      

## Combining across seasons

In [ ]:
# Filtering out latest week for current season (at time of export)
df_2021 = df_2021[df_2021['round'] < 12].copy()
max(df_2021['round'])

11

In [ ]:
# Retrieving column names from latest season
names = df_2021.columns
names

Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'value', 'transfers_balance', 'selected',
       'transfers_in', 'transfers_out', 'element_type', 'position',
       'opponent_strength'],
      dtype='object')

In [ ]:
# Filtering historic data to just columns available in latest data
df_1819_filtered = df_1819[names].copy()
df_1920_filtered = df_1920[names].copy()

In [ ]:
# Adding season labels
df_1819_filtered['season'] = 2018
df_1920_filtered['season'] = 2019
df_2021['season'] = 2020

In [ ]:
# Combining datasets
combined = pd.concat([df_1819_filtered, df_1920_filtered, df_2021]).copy()

## Checking new dataset before outputting

In [ ]:
# Checking new dataset
combined.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,element_type,position,opponent_strength,season
0,402,5,12,0,False,2018-08-12T12:30:00Z,4.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,55,0,103396,0,0,2,DEF,5,2018
1,402,250,12,2,True,2019-02-04T20:00:00Z,1.0,1.0,25,90,0,0,0,1,0,0,0,0,0,0,0,7,7.6,0.6,10.0,1.8,50,-787,32970,27,814,2,DEF,5,2018
2,145,15,12,-2,True,2018-08-20T19:00:00Z,0.0,2.0,2,74,0,0,0,2,0,0,0,0,1,0,0,-1,12.4,12.3,4.0,2.9,41,282492,1110097,296156,13664,2,DEF,5,2018
3,145,225,12,0,False,2019-01-19T15:00:00Z,4.0,3.0,23,90,0,0,0,4,0,0,0,0,0,0,0,15,17.8,0.8,0.0,1.9,45,21257,2025265,76053,54796,2,DEF,5,2018
4,386,128,12,1,True,2018-11-24T15:00:00Z,0.0,3.0,13,90,0,0,0,3,0,0,0,0,0,0,0,12,11.8,19.9,0.0,3.2,43,-768,6017,166,934,2,DEF,5,2018


In [ ]:
# Checking shape
combined.shape

(50545, 35)

In [ ]:
# Checking values are distinct at season, element, round and fixture level
combined.groupby(['season', 'element', 'round', 'fixture']).ngroups
# Note: as this mirrors the total amount of rows above I am satisfied that this is distinct at the right level

50545

In [ ]:
# Checking combined data types
combined.dtypes

element                int64
fixture                int64
opponent_team          int64
total_points           int64
was_home                bool
kickoff_time          object
team_h_score         float64
team_a_score         float64
round                  int64
minutes                int64
goals_scored           int64
assists                int64
clean_sheets           int64
goals_conceded         int64
own_goals              int64
penalties_saved        int64
penalties_missed       int64
yellow_cards           int64
red_cards              int64
saves                  int64
bonus                  int64
bps                    int64
influence            float64
creativity           float64
threat               float64
ict_index            float64
value                  int64
transfers_balance      int64
selected               int64
transfers_in           int64
transfers_out          int64
element_type           int64
position              object
opponent_strength      int64
season        

In [ ]:
#Downloading dataset
combined.to_csv('fpl_combined_data.csv',index=False) 
files.download('fpl_combined_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File downloaded to local machine and uploaded to GitHub